## 1. Import Library

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 126.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration

## 2. Data Load

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# path 수정
path= '/content/drive/My Drive/23 U 4-1/텍스트마이닝/기말프로젝트/Data/'
df_train= pd.read_csv(path+ 'train_cleansing.csv')
df_valid= pd.read_csv(path+ 'valid_cleansing.csv')

In [5]:
df_train.drop(['newsCategory', 'clickbaitClass', 'clickbaitTitle'], axis=1, inplace= True)
df_train.columns= ['target_text', 'input_text']
df_valid.drop(['newsCategory', 'clickbaitClass', 'clickbaitTitle'], axis=1, inplace= True)
df_valid.columns= ['target_text', 'input_text']

dfSummary= pd.DataFrame()
dfSummary['target_text']= df_valid['target_text']
dfSummary['context']= df_valid['input_text']

## 3. KOBART-Summary

*   시간이 아마 많이 오래 걸릴 겁니다,,, iRange(한번에 추상요약하는 수)를 적절히 조절해주시고, 중간중간 저장도 잘 해주시면 감사하겠습니다:>  
*   목표 개수: (앞에 1000개는 제가 했으므로 1000개부터) 10000개가 목표입니다!, 학습이 의외로 잘 돼서 너무 많이 할 필요는 없을 것 같습니다:>
*   (참고: 1000개 돌리는 데 약 7시간 정도 걸린 것 같습니다ㅠㅠ)

In [3]:
tokenizer= PreTrainedTokenizerFast.from_pretrained("gogamza/kobart-summarization")
model= BartForConditionalGeneration.from_pretrained("gogamza/kobart-summarization")

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [7]:
lzSummary= []
iStart= 0
iRange= 500 #한번에 추상요약하는 수, 소요시간에 따라 조정

for i in range(iRange):
  if len(dfSummary.iloc[iStart+i,1])>500: input_ids= tokenizer.encode(dfSummary.iloc[iStart+i,1][:500], return_tensors= "pt")
  else: input_ids= tokenizer.encode(dfSummary.iloc[iStart+i,1], return_tensors= "pt")
  
  summary= model.generate(
    input_ids= input_ids,
    bos_token_id= model.config.bos_token_id,
    eos_token_id= model.config.eos_token_id,
    length_penalty= .3, #bigger than 1= longer, smaller than 1=shorter summary
    max_length= 35,
    min_length= 25,
    num_beams= 5
  )
  lzSummary.append(tokenizer.decode(summary[0], skip_special_tokens= True))

In [6]:
# szTest= '배우 이이경 생방송 시상식중 모친과 전화 연결한 사연은? 저조한 시청률, 결국 정준하와 신봉선 하차 가수 미주와 배우 이이경이 열애설 및 데이트에 대한 가족들의 반응을 공개했습니다. 2023년 6월 10일 방송된 MBC 놀면 뭐하니?(이하 놀뭐)에서는 이이경과 미주가 지난 주 방송에 대한 가족들의 반응을 밝힌 장면이 그려졌습니다.이날 박진주는 이경 씨는 마음에 불편함이나 이런 건 없으시냐. 저번 미주와 데이트한 촬영으로 인해서라며 궁금해했습니다. 이에 유재석은 미주 부모님은 뭐라고 안 하시냐라며 거들었고, 미주는 "저 사실 이렇게 이야기하셨다. 너 이경이 진짜 아니야?" 이에 미주에 대한 답은 "아니야"라고 말했습니다.이이경은 "어머니한테 카톡 왔다. 방송 끝나자마자 어머니한테 카톡 왔다. 너 죽어도 개띠는 안 된다"라며 미주와 연애를 반대했다는 사실을 고백했습니다.배우 이이경 생방송 시상식중 모친과 전화 연결한 사연은?한편 배우 이이경은 즉석 전화연결 수상 소감으로 웃음과 감동을 동시에 안겼던 바 있습니다. 2021 KBS 연기대상"에서 이이경은 남자 조연상 수상자로 호명됐습니다. 이이경은 2TV 드라마 "암행어사: 조선비밀수사단"에 박춘삼 역으로 출연해 극에 재미를 더한 바 있습니다.트로피를 받은 이이경은 연기한 지 10년 됐는데 그동안 누군가에게 상을 주기만 해봤다. 받는 건 처음이라며 기뻐했습니다. 이어 그는 아무런 기억이 나지 않는다고 하더니 보고 싶은 사람에게 전화연결 한 번 하겠다는 깜짝 선언을 했습니다. 이이경을 전화 연결이 되자 자기야!라고 말하며 궁금증을 높였습니다. 알고 보니 통화 상대는 이이경의 어머니였는데 이이경은 모친이 지금 방송 보고 있다고 답하자 자기야, 너무 오래 걸렸지. 아들 상 받았어라고 답했습니다. 이이경의 모친은 좋은 결과 있어서 엄마 아빠가 너무 좋다. 즐거움을 주는 배우가 됐으면 한다고 축하 인사를 전했습니다. 감격의 눈물을 보이기도 한 이이경은 내일 트로피 들고 집에 들어가겠다. 생방송이라 이만 끊겠다고 한 뒤 통화를 마무리했습니다. 끝으로 그는 암행어사에서 호흡한 김명수, 권나라에게 감사 인사를 하며 신박했던 수상 소감을 깔끔하게 마무리했습니다.저조한 시청률, 결국 정준하와 신봉선 하차한편 방송 말미, 유재석은 놀면뭐하니가 2주간의 재정비 시간을 가지게 됐다라고 운을 떼며 마음이 좀 많이 무겁고 아프다. 그동안 우리와 함께 해줬던 미나와 준하형이 마지막으로 녹화를 하게됐다라고 전했습니다.정준하는 더 좋은 모습을 보여드리지 못해 죄송스러운 마음이 들고, 놀면 뭐하니도 앞으로 승승장구 하기 위해 좋은 에너지가 필요하다고 생각한다"라고 덤덤하게 하차 심경을 전했습니다. 신봉선은 눈물을 보이며 가볍게 가고 싶다. 남아있는 사람들도 마음 편했으면 좋겠고, 가는 우리도 마음 편했으면 좋겠다고 말했습니다.또한 정준하는 지금 저희는 행복하니까 걱정 마시고, 우리는 가족이니까 필요하시면 언제든 불러달라고 말했고, 유재석은 앞으로 2주간 재정비 시간을 가지고 더큰 웃음을 가지고 돌아오겠다라고 덧붙였습니다.MBC 놀면 뭐하니?는 2주간의 재정비 시간을 가진 후 오는 7월 1일 돌아올 예정입니다.'

# if len(szTest)>500: input_ids= tokenizer.encode(szTest[:500], return_tensors= "pt")
# else: input_ids= tokenizer.encode(dfSummary.iloc[iStart+i,1], return_tensors= "pt")

# summary= model.generate(
#   input_ids= input_ids,
#   bos_token_id= model.config.bos_token_id,
#   eos_token_id= model.config.eos_token_id,
#   length_penalty= .3, #bigger than 1= longer, smaller than 1=shorter summary
#   max_length= 35,
#   min_length= 25,
#   num_beams= 5)

In [8]:
tokenizer.decode(summary[0], skip_special_tokens= True)

'2023년 6월 10일 방송된 MBC 놀면 뭐하니?(이하 놀뭐)에서는 이이경과 이이경이 열애설 및 데이트에 대한 가족들의 반응을 공개'

## 3-2. Summary Save

In [8]:
df_Final= pd.DataFrame()
df_Final['Summary']= lzSummary
df_Final['title']= list(dfSummary[iStart:iStart+iRange]['target_text'])

In [9]:
df_Final.to_csv('valid_summ.csv', encoding= 'utf-8-sig', index= False)